# Data cleaning and merging dataframes

## Loading multiple datasets

**Google way**

In [67]:
import numpy as np

In [8]:
# import pandas as pd

# # orderlines.csv
# url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' 
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# orderlines = pd.read_csv(path)

# # orders.csv
# url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' 
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# orders = pd.read_csv(path)

# print(path)

# # brands.csv
# url = 'https://drive.google.com/file/d/1BrNrIY0F1LbyXtyaQygUBXVxQGB3JBqx/view?usp=sharing' 
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# brands = pd.read_csv(path)

# # products.csv
# url = 'https://drive.google.com/file/d/1UfsHI80cpQqGfsH2g4T4Tsw8cWayOfzC/view?usp=sharing' 
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# products = pd.read_csv(path)

# df_list = [orderlines, orders, brands, products]
# files = ['orderlines','orders','brands','products']



**Other ways**

In [9]:
import pandas as pd

path = './data/eniac/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

In [ ]:
# import pandas as pd
# import os
# path = '../data/eniac/'
# path, dirs, files = next(os.walk(path))
# #print(files)

# # remove non-csv files
# for file in files:
#     if not file.endswith("csv"):
#         files.remove(file)

# # create empty list 
# df_list = []

# # append datasets to the list
# for file in files:
#     temp_df = pd.read_csv(path + file, sep=',')
#     df_list.append(temp_df)

# # show results
# products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [13]:
#check missing values for the whole dataframe
orderlines.isna().sum().sum()

0

In [14]:
# check missing values column
orderlines.isna().sum()

id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

In [16]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [17]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [18]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [19]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Duplicates

The presence of duplicate rows is generally a sign that the data is not clean, and will deserve further exploration.

In [20]:
orderlines.duplicated().sum()

0

In [21]:
orders.duplicated().sum()

0

In [22]:
brands.duplicated().sum()

0

In [23]:
products.duplicated().sum()

8746

In [24]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Data cleanliness - initial assessment

Based on our initial exploration, we know we will need to deal with some missing values. The biggest issue so far are the duplicates on the `products` DataFrame. Here are some aspects that we will need to fix or, at least, explore further:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 
    * Duplicates: a total of 8746 duplicates seems to indicate this DataFrame has been seriously corrupted.
    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` matches with the created dates for `orderlines`. 
    
* **orderlines**: 
    * `unit_price` has to be a float, something is wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it matches with the `orders` dataset. 
    
* **brands**: looks fine. 

Where do we have to start? 

1. **Data consistency:** Since `orders` and `orderlines` seem very crucial to the analysis, we will start by cleaning them and checking that the information present in both DataFrames match.

2. **The "products mess":** This file seems to have many issues. We will leave it out for now and perform a proper exploration later to understand better what's going on there.

## Cleaning orders

The data consistency check we will do with `orderlines` will involve two steps: 

* the initial and last dates of the orders should be the same
* the sum of `total_paid` on both datasets should be the same

Let's start by transforming the `created_date` of the orders DataFrame and looking for its earliest and latest values:

In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [26]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

In [27]:
# earliest value
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [28]:
# latest value
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

Now we will look at the overall sum of `total_paid` for the orders table:

In [29]:
sum(orders['total_paid'])

nan

the result of the sum is a nan (not a number).
here are missing values! We can explore them and see how they are all "Pending" orders:

In [30]:
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


its not necessary, but i want to search the orderliness to find if this missed orders exist. then i calculate the total_price from the orderlines to this missed datas

In [32]:
missed_orders =(
orders
      .loc[orders['total_paid'].isna()]
)

missed_orders.assign(check_orders = missed_orders['order_id'].isin(orderlines['id_order']))

,order_id,created_date,total_paid,state,check_orders
127701,427314,2017-11-20 18:54:39,NaN,Pending,True
132013,431655,2017-11-22 12:15:24,NaN,Pending,True
147316,447411,2017-11-27 10:32:37,NaN,Pending,True
148833,448966,2017-11-27 18:54:15,NaN,Pending,True
149434,449596,2017-11-27 21:52:08,NaN,Pending,True


we can recalculate all 5 missed values from orderlines, lets to calculate them 

In [33]:
missed_orders_total_paid = (
missed_orders
            .merge(
                    orderlines, 
                    how= 'inner', 
                    left_on= 'order_id',
                    right_on= 'id_order'
                  )
            .assign(unit_price_new = lambda df_ : df_['unit_price'].replace(r'(\d*)\.(\d{3}\.\d{2})$', r'\1\2', regex= True).astype('float'))
            .groupby('order_id')
            .agg(repaired_total_price = ('unit_price_new', 'sum'))
            .reset_index()
)
missed_orders_total_paid 

,order_id,repaired_total_price
0,427314,75.99
1,431655,154.00
2,447411,180.76
3,448966,961.02
4,449596,1722.59


and now ill fill the missed values with my new values

In [34]:
def get_total_price(order):
  result = (
            missed_orders_total_paid
           .merge(
                  order,
                  how= 'right',
                  left_on= 'order_id',
                  right_on= 'order_id'
                  )
           .loc[:,'repaired_total_price']
  )
  return result

orders = orders.fillna(value ={"total_paid" : get_total_price(orders['order_id']) })

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226909 non-null  int64         
 1   created_date  226909 non-null  datetime64[ns]
 2   total_paid    226909 non-null  float64       
 3   state         226909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.9+ MB


**extra** : Since these orders are only a tiny fraction and there's a valid reason why the total_paid value is missing, we could simply exclude them from the dataset.

In [ ]:
# orders.dropna(inplace=True)

Now the dataset is clean. And the total paid is: 

In [35]:
sum(orders['total_paid'])

129162709.42991276

## Cleaning orderlines

Following our data consistency check, will now gather in the orderlines DataFrame the same information we got from orders:

* the initial and last dates
* the sum of `total_paid`


First let's transform our date time:

In [36]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [37]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [38]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

**It's a match!**

Now let's check the `total_paid` for orderlines:

In [39]:
orderlines.head(3)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57


To get this value, we will have to calculate a new column, total price for each row. It would be product_quantity * unit_price. This operation will require that both columns have a numeric data type:

In [40]:
orderlines.dtypes

id                           int64
id_order                     int64
product_id                   int64
product_quantity             int64
sku                         object
unit_price                  object
date                datetime64[ns]
dtype: object

...it's not the case right now, so we will have to transform the unit_price to a numeric data type.


In [ ]:
# uncomment the line of code below and read the error it produces:
# orderlines['product_quantity']*pd.to_numeric(orderlines['unit_price'])

While trying to transform this column to numeric an error appears. From the error message,

"Unable to parse string '1.137.99' at position 6"

it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and Python & pandas only admit one dot per number: the decimal separator!

Lesson learned: do not use thousand separators in databases / statistical software / programming languages! Sadly, it's too late for us, and we will have to deal with the issue.

There are many ways to approach this problem.

i use a replace regex pattern to remove the thousand point, and then i convert the unit price to float. and add the new values in a new column "new_unit_price"

In [44]:
orderlines= (
orderlines    
        .assign(new_unit_price = lambda df_ : df_['unit_price'].replace(r'(\d*)\.(\d{3}\.\d{2})$', r'\1\2', regex= True).astype('float'))
)

In [41]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


Data cleaning done! 

Back to data consistency: Now it is time to multiply `product_quantity` and `unit price`, sum all the rows and check whether the value matches the sum of the `total_paid` from the orders DataFrame: 

In [49]:
# create a new column "total_price" by multiplying product_quantity times unit_price
orderlines =(
orderlines
        .assign(total_price = lambda df_ : df_['product_quantity'] * df_['new_unit_price'])
)

# sum of the new column "total_price":
orderlines['total_price'].sum()

128776222.02999999

Sadly, it does not match exactly with the sum of `total_paid` from orders:

In [48]:
orders['total_paid'].sum()

129162709.43

The mismatch is about 386 thousand dollars, a non-neglegible amount of money:

In [50]:
orderlines['total_price'].sum() - orders['total_paid'].sum()

-386487.40000002086

## Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of its major problems: 

In [53]:
print(products.info(), "\n")
print("Missing values:", products.isna().sum(), "\n")
print("Duplicate rows:", products.duplicated().sum())
print("Duplicate rows:", products.duplicated(subset='sku').sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None 

Missing values: sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64 

Duplicate rows: 8746
Duplicate rows: 8747


Looking at this overview, we can see that there are different things that have to be changed: 

* Data types: 
    * `price` should be a float
    * `promo price` should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description maybe can be inferred by the name
    * `price`. Is there a way we can extract the information from another table?
    * `type`. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

*Duplicates*

In [56]:
products.drop_duplicates(subset= 'sku', inplace= True)

*Deal with the "price" column*

at first step, we will check if the **missed** price are exist in orderlines, that means are they sold and important to us.
we just chek the missed prices sku if is in orderlines sku s

In [85]:

(
    products
            .loc[lambda df_ : df_['price'].isna()]
            .loc[lambda df_ : df_['sku'].isin(orderlines['sku'])]
)

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,0,11865403
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,0,11865403
2049,CEL0015,Celly fur-lined Powerbank battery 4000mAh Black,Leather-wrapped External Battery 4000mAh for i...,NaN,239.895,0,1515
2052,CEL0023,Celly Ambo Luxury Leather Case + Case Gold iPh...,Cover and housing together with magnet for iPh...,NaN,329.894,0,11865403
12052,CEL0028,Celly Swing Arm button selfie with Bluetooth B...,selfie extendable arm for iPhone,NaN,59.895,0,11905404
12053,CEL0029,Celly extensible arm selfie with Bluetooth but...,selfie extendable arm for iPhone,NaN,69.902,0,5720
12054,CEL0030,Celly Smart Drive Housing + clip car holder iP...,Magnetic holder for car Compatible iPhone 5 5s...,NaN,89.903,0,5720
12061,CEL0043,Celly Fluor Case iPhone 6 / 6S Pink / Transparent,Protective Case for iPhone 6 and 6s,NaN,99.898,0,11865403
12062,CEL0044,Celly iPhone Case Fluor 6s / 6 Yellow / Transp...,protective case with two colors for iPhone 6 a...,NaN,79.896,1,11865403


the rows that didnt appear in orderlines, they didnt sold at all, and we can delete them, but the others we can easilly recalculate the missed value for them.
for this i will give the recoverable missed value 0, and then use dropna() for delete the others

In [98]:
recoverable_missed_price =(
    products
            .loc[lambda df_ : df_['price'].isna()]
            .loc[lambda df_:df_['sku'].isin(orderlines['sku'])]
)

products = (
products
        .assign(new_price =np.where(
                                    products['sku'].isin(recoverable_missed_price['sku']),
                                    "0", products['price']
                                )
                )
)

products.dropna(subset='new_price', inplace= True)
# products
recoverable_missed_price

,sku,name,desc,price,promo_price,in_stock,type,new_price
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696,0
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,0,11865403,0
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,0,11865403,0
2049,CEL0015,Celly fur-lined Powerbank battery 4000mAh Black,Leather-wrapped External Battery 4000mAh for i...,NaN,239.895,0,1515,0
2052,CEL0023,Celly Ambo Luxury Leather Case + Case Gold iPh...,Cover and housing together with magnet for iPh...,NaN,329.894,0,11865403,0
12052,CEL0028,Celly Swing Arm button selfie with Bluetooth B...,selfie extendable arm for iPhone,NaN,59.895,0,11905404,0
12053,CEL0029,Celly extensible arm selfie with Bluetooth but...,selfie extendable arm for iPhone,NaN,69.902,0,5720,0
12054,CEL0030,Celly Smart Drive Housing + clip car holder iP...,Magnetic holder for car Compatible iPhone 5 5s...,NaN,89.903,0,5720,0
12061,CEL0043,Celly Fluor Case iPhone 6 / 6S Pink / Transparent,Protective Case for iPhone 6 and 6s,NaN,99.898,0,11865403,0
12062,CEL0044,Celly iPhone Case Fluor 6s / 6 Yellow / Transp...,protective case with two colors for iPhone 6 a...,NaN,79.896,1,11865403,0


now we can recalculate the missed prices with the max price for that product in orderlines.


In [109]:

recoverd_values = (
orderlines
        .groupby('sku')
        .agg(max_price = ('unit_price', 'max'))
        .reset_index()
        .merge(
                products,
                how= 'right',
                on='sku'
        )                           
)

In [112]:
recoverd_products=(
products
        .assign(recovered_price_1 =np.where(
                                    products['new_price']=="0",
                                    recoverd_values['max_price'], products['new_price']
                                )
                )
)
recoverd_products

,sku,name,desc,price,promo_price,in_stock,type,new_price,recovered_price_1
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,59.99,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,59,59
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,59,59
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,25,25
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,34.99,34.99
...,...,...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282,29.99,29.99
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,69.95,69.95
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,69.95,69.95
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392,69.95,69.95
